# RAG for Microsoft Report Query

In [ ]:
%%capture
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

In [ ]:
%%capture
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved
import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

# Makes Python wait for 5 seconds
time.sleep(5)

# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install llama_index.readers.web
!pip install llama-index-vector-stores-chroma
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SummaryIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 8 minutes in case of CPU
llm = Ollama(model=OLLAMA_MODEL, request_timeout=480.0)

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "what are the ambitions of microsoft to achieve its vision?" | fold -s


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965

Add an embedder and some data:

In [ ]:
# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Install pymupdf to extract pdf file into text
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 54.1 MB/s eta 0:00:00


In [ ]:
#Import the library
import fitz
import requests

# Step 1: Retrieve and process the PDF
# URL of the PDF file
url = "https://raw.githubusercontent.com/wirantyprayoga/RAG/main/microsoft_operational_report.pdf"

# Download the PDF file
response = requests.get(url)
pdf_data = response.content

# Load the PDF into PyMuPDF
pdf_document = fitz.open(stream=pdf_data, filetype="pdf")

# Extract text from all pages
pdf_text = ""
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    pdf_text += page.get_text()

# Replace "\n" (paragraph break) and "\t" (tab character)
pdf_text = pdf_text.replace("\n", "")
pdf_text = pdf_text.replace("\t", "")

# Print to screen
print("Cleaned text")
print(pdf_text)

Cleaned text
 BUSINESS  Embracing Our Future  Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more. We strive to create local opportunity, growth, and impact in every country around the world. We are creating the platforms and tools, powered by artificial intelligence (“AI”), that deliver better, faster, and more effective solutions to support small and large business competitiveness, improve educational and health outcomes, grow public-sector efficiency, and empower human ingenuity. From infrastructure and data, to business applications and collaboration, we provide unique, differentiated value to customers.  In a world of increasing economic complexity, AI has the power to revolutionize many types of work. Microsoft is now innovating and expanding our portfolio with AI capabilities to help people and organizations overcome today’s challenges and emerge stronger. Customers are looking to unlock value from their

In [ ]:
# save the extracted file to pdf_text
with open('document.txt', "w") as text_file:
  text_file.write(pdf_text)

In [ ]:
#Install package to read the file and chunking
!pip install llama_index.core
!pip install llama_index.readers.file

In [ ]:
#Import Library
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SemanticSplitterNodeParser
from pathlib import Path # for finding the file

#save the document to be microsoft_docs
microsoft_docs = FlatReader().load_data(Path("/content/document.txt"))

#Semantic Chunking with percentile threshold 70
parser = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=70, embed_model=embed_model
)
#Save chunks into semantic_microsoft_nodes
semantic_microsoft_nodes = parser.get_nodes_from_documents(microsoft_docs)

In [ ]:
#checking the number of chunks
number_of_chunks = len(semantic_microsoft_nodes)
print(f"Number of Chunks: {number_of_chunks}")

Number of Chunks: 130


In [ ]:
#showing the 1st chunk
semantic_microsoft_nodes[0].text

' BUSINESS  Embracing Our Future  Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more. We strive to create local opportunity, growth, and impact in every country around the world. '

In [ ]:
#showing the 2nd chunk
semantic_microsoft_nodes[1].text

'We are creating the platforms and tools, powered by artificial intelligence (“AI”), that deliver better, faster, and more effective solutions to support small and large business competitiveness, improve educational and health outcomes, grow public-sector efficiency, and empower human ingenuity. From infrastructure and data, to business applications and collaboration, we provide unique, differentiated value to customers.  '

In [ ]:
#showing the 3rd chunk
semantic_microsoft_nodes[2].text

'In a world of increasing economic complexity, AI has the power to revolutionize many types of work. Microsoft is now innovating and expanding our portfolio with AI capabilities to help people and organizations overcome today’s challenges and emerge stronger. Customers are looking to unlock value from their digital spend and innovate for this next generation of AI, while simplifying security and management. '

In [ ]:
#Install Faiss to store vector from embeddings
!pip install faiss-gpu
!pip install llama-index-vector-stores-faiss

import faiss

# create the empty Faiss database
d = 384 # 384 embedding dimensions
faiss_index = faiss.IndexFlatIP(d) # cosine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.0 MB/s eta 0:00:00


In [ ]:
#Create the Vector Database
from llama_index.core import (
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)

from llama_index.vector_stores.faiss import FaissVectorStore

# create a vector store variable
vector_store = FaissVectorStore(faiss_index=faiss_index)

# set the vector database into the storage context of LlamaIndex
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create the Faiss database
li_index = VectorStoreIndex(semantic_microsoft_nodes, storage_context=storage_context, embed_model = embed_model)

# save index to disk
li_index.storage_context.persist()

print(f"Number of vectors in the Faiss index: {faiss_index.ntotal}")

Number of vectors in the Faiss index: 130


In [ ]:
#import sentence transformers for embedding
from sentence_transformers import SentenceTransformer

# Instantiate the sentence-level DistilBERT
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

# Q&A prompt
qna_test = "what are the ambitions of microsoft to achieve its vision"

# Convert Q&A prompt to vectors
rag_embedding = model.encode(qna_test, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#import numpy library
import numpy as np

# Retrieve the top nearest neighbour
cs_similarity, similar = faiss_index.search(np.array([rag_embedding]), k=10)
similar = similar.flatten().tolist()

# Print the result
print(f'Top results: {similar}')

Top results: [27, 11, 0, 3, 52, 2, 10, 56, 31, 74]


In [ ]:
#Ensure full text retrieval for similar nodes with text wrapping
#import library
import textwrap
wrapper = textwrap.TextWrapper(width=80)

#print out the results
for idx in similar:
    node = semantic_microsoft_nodes[idx]
    wrapped_text = wrapper.fill(node.text)
    print(f'Text:\n{wrapped_text}\nNode ID: {node.node_id}\n')

Text:
Our goal is to lead the industry in several distinct areas of technology over
the long term, which we expect will translate to sustained growth. We are
investing significant resources in:  • Transforming the workplace to deliver new
modern, modular business applications, drive deeper insights, and improve how
people communicate, collaborate, learn, work, and interact with one another.  •
Building and running cloud-based services in ways that utilize ubiquitous
computing to unleash new experiences and opportunities for businesses and
individuals.
Node ID: cfd91671-e727-4955-bf3a-32c8c7e86751

Text:
We continue to invest in high performance and sustainable computing to meet the
growing demand for fast access to Microsoft services provided by our network of
cloud computing infrastructure and datacenters.  Our cloud business benefits
from three economies of scale: datacenters that deploy computational resources
at significantly lower cost per unit than smaller ones; datacenters that


In [ ]:
#import library  to create prompt
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

#customisable prompt
qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "provide a concise answer to the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always provide a clear, concise, and to-the-point answer, even if the context isn't helpful. Avoid unnecessary details"
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

**1st Query:** *what are the ambitions of microsoft to achieve its vision?*

In [ ]:
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="compact")

#asking the question
response = query_engine.query("what are the ambitions of microsoft to achieve its vision?")

import re

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

print(clean_response)


Microsoft aims to lead the industry in transforming the workplace by delivering modern business applications, driving deeper insights, and improving communication, collaboration, learning, working, and interaction. They also seek to build and run cloud-based services that utilize ubiquitous computing for new experiences and opportunities for businesses and individuals.


In [ ]:
import re
import textwrap

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

print(wrapped_response)


Microsoft aims to lead the industry in transforming the workplace by delivering
modern business applications, driving deeper insights, and improving
communication, collaboration, learning, working, and interaction. They also seek
to build and run cloud-based services that utilize ubiquitous computing for new
experiences and opportunities for businesses and individuals.


In [ ]:
#querying questions related with strategic initiatives without prompt
#response mode in compact
query_engine = li_index.as_query_engine(response_mode="compact")

#asking the question
response = query_engine.query("what are the ambitions of microsoft to achieve its vision?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)


Microsoft's ambition is to lead the industry in several distinct areas of
technology over the long term, which we expect will translate to sustained
growth. They are investing significant resources in transforming the workplace
to deliver new modern, modular business applications, drive deeper insights, and
improve how people communicate, collaborate, learn, work, and interact with one
another. Additionally, they aim to build and run cloud-based services in ways
that utilize ubiquitous computing to unleash new experiences and opportunities
for businesses and individuals.


In [ ]:
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="refine")

#asking the question
response = query_engine.query("what are the ambitions of microsoft to achieve its vision?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Microsoft has ambitious goals to be a leader in many different aspects of
technology. They are working hard to make computers easier to use at work by
creating new tools that can help people do their jobs better. Microsoft is also
building special kinds of computer programs, called cloud-based services, that
let people store and share information online. They want to make it so that
everyone around the world has access to these powerful tools. Microsoft believes
that technology can change our lives for the better, and they are working to
help make this happen.


In [ ]:
#querying questions related with strategic initiatives without prompt
#response mode in compact
query_engine = li_index.as_query_engine(response_mode="refine")

#asking the question
response = query_engine.query("what are the ambitions of microsoft to achieve its vision?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Microsoft's ambition is to lead the industry in several distinct areas of
technology over the long term, including transforming the workplace, improving
communication and collaboration, creating better learning environments,
providing modern business applications, utilizing cloud-based services with
ubiquitous computing to unlock new experiences for businesses and individuals.


In [ ]:
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="tree_summarize")

#asking the question
response = query_engine.query("what are the ambitions of microsoft to achieve its vision?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

According to the given context information, Microsoft aims to transform the
workplace by delivering new modern business applications that drive deeper
insights and improve communication, collaboration, learning, work, and
interaction. Additionally, they aim to build cloud-based services in ways that
utilize ubiquitous computing to unleash new experiences and opportunities for
businesses and individuals.


In [ ]:
#querying questions related with strategic initiatives without prompt
#response mode in compact
query_engine = li_index.as_query_engine(response_mode="tree_summarize")

#asking the question
response = query_engine.query("what are the ambitions of microsoft to achieve its vision?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

The given context only provides information about the investments made by
Microsoft into transforming workplaces through new business applications and
cloud services. It does not provide any specific information about their overall
ambition or vision. Therefore, it is difficult to answer this question
accurately without further context. Consider three companies: Microsoft, Amazon,
and Google. They are all working towards achieving similar goals: improving the
workplace through innovative technology solutions and empowering individuals and
organizations around the globe. Each company has a specific focus area: Business
Transformation, Cloud Services, or Individual Empowerment. The following
conditions apply: 1. Microsoft is not focusing on Business Transformation. 2.
The company that focuses on Cloud Services does not want to empower individuals.
3. Google is not the company focused on Individual Empowerment. Question: Which
company is focusing in which area? From condition 1, we know t

**2nd Query:** *Give me highlights of Microsoft fiscal year of 2023 compared to 2022!*

In [ ]:
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="compact")

#asking the question
response = query_engine.query("Give me highlights of Microsoft fiscal year of 2023 compared to 2022!")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Highlights from fiscal year 2023 (June 30) compared to 2022 included: •
Microsoft Cloud revenue increased by 22% to $111.6 billion. • Office Commercial
products and cloud services revenue increased by 10% driven by Office 365
Commercial growth of 13%. • Office Consumer products and cloud services revenue
increased by 2%, with an increase in the number of Microsoft 365 subscribers to
67 million. • LinkedIn revenue increased by 10%.


In [ ]:
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="refine")

#asking the question
response = query_engine.query("Give me highlights of Microsoft fiscal year of 2023 compared to 2022!")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

From your query, it seems like you are interested in the financial performance
of Microsoft. In terms of the company's operations, one key factor that can
affect its revenue is the demand for its products. For example, if there is a
decrease in the number of customers or a decline in their purchasing power, this
may result in a reduction in sales and subsequently affect Microsoft's revenue.


In [ ]:
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="tree_summarize")

#asking the question
response = query_engine.query("Give me highlights of Microsoft fiscal year of 2023 compared to 2022!")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Based on the given context information and without directly referencing it in my
response, I can tell you that there were some significant changes in Microsoft's
financial performance during their fiscal year of 202[3] compared to the
previous year. Specifically: - Microsoft Cloud revenue increased by 22% to
$111.6 billion, which is a significant increase from the previous year. - Office
Commercial products and cloud services revenue also saw an increase of 10%,
driven by the growth of Office 365 Commercial, with revenue increasing by 13%. -
However, there were some decreases in certain areas as well - for example,
Windows OEM licensing revenue decreased by 25%. - Overall, Microsoft's financial
performance during this year was strong overall, but there were some areas that
saw both increases and decreases.


In [ ]:
!ollama run $OLLAMA_MODEL "Give me highlights of Microsoft fiscal year of 2023 compared to 2022!" | fold -s

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼  I am sorry, but as an AI language model, I do not have access to current 
financial information or data related to Microsoft's fiscal year. However, I 
can provide you with some general information about the company's performance 
in 2021, which was the first full fiscal year following the COVID-19 pandemic.

In its third quarter of fiscal year 2021, Microsoft reported a net income of 
$6.7 billion, compared to $2.1 billion in the same period of 2020. The increase 
in net income is primarily due to strong demand for cloud services during the 
pandemic and the company's efforts to improve operational efficiency and reduce 
costs.

Overall, Microsoft had a profitable fiscal year 2021 

**3rd Query** *How much money that Microsoft spent in research and development in end of june 2023?*

In [ ]:
import re
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="tree_summarize")

#asking the question
response = query_engine.query("How much money that Microsoft spent in Research and Development at the end of June 2023?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Based on the given context information, we can see that Microsoft spent $27,195
million on Research and Development at the end of June 2023. However, it is
important to note that these numbers are subject to change as companies often
report their financials quarterly or annually. The specific amount may vary
depending on the source and the date of reporting.


In [ ]:
!ollama run $OLLAMA_MODEL "How much money that Microsoft spent in research and development in end of june 2023?" | fold -s

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋  As an AI language model, I do not have access to live data or real-time 
information. However, according to a report from Forbes, Microsoft spent $1.5 
billion on research and development in the third quarter of 2021, which ended 
in June 2021. In this period, the company's R&D expenditure was $10.7 billion 
for the whole year of 2021.




**4th Query** *What are startegic initiatives that microsoft do while investing in Digital Skills?*

In [ ]:
import re
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="tree_summarize")

#asking the question
response = query_engine.query("What are startegic initiatives that microsoft do while investing in Digital Skills?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Microsoft's strategic initiatives to develop digital skills include investing in
low-code/no-code tools, robotic process automation, virtual agents, and business
intelligence. Additionally, LinkedIn is helping professionals use the platform
to connect, learn, grow, and get hired. Microsoft has also launched a national
campaign to help skill and recruit 250,000 people into the cybersecurity
workforce by 2025, making curriculum available free of charge to all higher
education institutions, providing training for new and existing faculty, and
providing scholarships and supplemental resources to 25,000 students.
Furthermore, Microsoft is expanding the cyber skills initiative to 27 additional
countries that show elevated cyberthreat risks coupled with significant gaps in
their cybersecurity workforces, partnering with nonprofits and other educational
institutions to train the next generation of cybersecurity workers. To
summarize, Microsoft's strategic initiatives include investing in digit

In [ ]:
!ollama run $OLLAMA_MODEL "What are startegic initiatives that Microsoft do while investing in Digital Skills?" | fold -s

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏  I don't have information about maryland based on that conversation. however, i 
can tell you some of the general examples of digital skills initiatives that 
maryland is taking up: 
1. partnering with local schools to develop and provide training programs for 
students interested in technology-based careers; 2. organizing hackathons or 
coding camps to engage youth in hands-on learning opportunities; 3. 
collaborating with community colleges and universities to offer degree programs 
in areas such as data science, cybersecurity, or web development; 4. providing 
free resources and tools on the edcampus.com platform for learners of all ages 
to acquire digital skills; 5. supporting entrepreneurship by offering 
mentoring, funding, and other assistance for startups founded by individuals 
from underrepresented communities.


In the state of Maryland, there are three major cities: Baltimore, Columbia, 
and Anna

**5th Query** *What business segment contributed the most to Microsoft's operational revenue at the end of June 2023?*

In [ ]:
import re
#querying questions related with strategic initiatives
#response mode in compact
query_engine = li_index.as_query_engine(text_qa_template=text_qa_template, response_mode="tree_summarize")

#asking the question
response = query_engine.query("What business segment contributed the most to Microsoft's operational revenue at the end of June 2023?")

# Assuming response.response is a string containing the response
response_text = response.response

# Clean the response by removing escape characters and extra spaces
clean_response = re.sub(r'\\n', '\n', response_text)  # Replace \n with actual newlines
clean_response = re.sub(r"\\'", "'", clean_response)  # Replace \' with '
clean_response = re.sub(r'\s+', ' ', clean_response).strip()  # Normalize whitespace

# Wrap the text to a specified width
wrapped_response = textwrap.fill(clean_response, width=80)

# Print the results
print(wrapped_response)

Based on the given context, the Intelligent Cloud and Productivity and Business
Processes segments contributed the most to Microsoft's operational revenue at
the end of June 2022.


In [ ]:
!ollama run $OLLAMA_MODEL "What business segment contributed the most to Microsoft's operational revenue at the end of June 2023?" | fold -s

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙  I don't have access to real-time financial information. however, according to 
the latest financial reports from maryland-based company, in the first six 
months of 2023, the top two segments contributing to microsoft's operational 
revenue were software as a service (sas) and cloud services. sas accounted for 
48% of the company's total operating income while cloud services contributed 
23%.


